In [50]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [51]:
import os
num=[]
Z = []
path = '/Users/liuyingjie/Downloads/data/multi/3/'
filenames = os.listdir(path)
indexs = []
for filename in filenames:
    if filename.startswith("."):
        continue
    indexs.append(int(filename))
indexs.sort()
#print(indexs)
for index in indexs:
    file_path = path + str(index)
    with open(file_path,'r',encoding='utf8') as f:
        lines=f.readlines()
        #print(lines)
    for i in range(102, 122):
        temps = lines[i].split()
        #print(temps)
        point=[]
        point.append(float(temps[2])/100)
        point.append(float(temps[3])/100)
        num.append(point)
    for line in lines:
        temps = line.split()
        size = len(temps)
        #print(size)
        if(size == 11 and temps[0]=='10000000'):
            #print(temps)
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
        if(size == 16 and temps[0] == '10000000'):
            #print(temps)
            Z.append(int(float(temps[1])*37000000000))
            Z.append(int(float(temps[6])*37000000000))
            Z.append(int(float(temps[11])*37000000000))
            
for i in range(len(Z)):
    num[i].append(Z[i])
print(num)

[[90.0, 90.0, 2017], [91.0, 90.0, 2353], [92.0, 90.0, 2804], [93.0, 90.0, 3444], [94.0, 90.0, 4369], [95.0, 90.0, 5691], [96.0, 90.0, 7634], [97.0, 90.0, 10363], [98.0, 90.0, 13312], [99.0, 90.0, 14774], [100.0, 90.0, 13361], [101.0, 90.0, 10519], [102.0, 90.0, 7888], [103.0, 90.0, 5992], [104.0, 90.0, 4740], [105.0, 90.0, 3900], [106.0, 90.0, 3301], [107.0, 90.0, 2962], [108.0, 90.0, 2559], [109.0, 90.0, 2286], [110.0, 90.0, 2062], [110.0, 91.0, 2295], [110.0, 92.0, 2551], [110.0, 93.0, 2887], [110.0, 94.0, 3313], [110.0, 95.0, 3904], [110.0, 96.0, 4751], [110.0, 97.0, 6020], [110.0, 98.0, 7906], [110.0, 99.0, 10531], [110.0, 100.0, 13410], [110.0, 101.0, 14798], [110.0, 102.0, 13339], [110.0, 103.0, 10398], [110.0, 104.0, 7688], [110.0, 105.0, 5732], [110.0, 106.0, 4400], [110.0, 107.0, 3498], [110.0, 108.0, 2860], [110.0, 109.0, 2413], [110.0, 110.0, 2102], [109.0, 110.0, 2226], [108.0, 110.0, 2357], [107.0, 110.0, 2484], [106.0, 110.0, 2583], [105.0, 110.0, 2679], [104.0, 110.0, 27

In [61]:
import numpy as np
from numpy.linalg import solve
def createRegionByFirstPoint(point,extendTimes,interval):  #单点坐标
    pointStart=[point[0]-extendTimes*interval,point[1]+extendTimes*interval]
    pointEnd=[point[0]+extendTimes*interval,point[1]-extendTimes*interval]
    ans=[]
    pointY,pointX=pointStart[1],pointStart[0]
    while pointY>=pointEnd[1]:
        pointX=pointStart[0]
        while pointX<=pointEnd[0]:
            res=[pointX,pointY]
            ans.append(res)
            pointX+=interval
        pointY-=interval
    return ans

def createRegionBySecondPoint(point,extendTimes,interval,res):  #单点坐标
    pointStart=[point[0]-extendTimes*interval,point[1]+extendTimes*interval]
    pointEnd=[point[0]+extendTimes*interval,point[1]-extendTimes*interval]
    pointY,pointX=pointStart[1],pointStart[0]
    ans=[]
    while pointY>=pointEnd[1]:
        pointX=pointStart[0]
        while pointX<=pointEnd[0]:
            for r in res:
                if getDistance(r,[pointX,pointY]) >25:
                    pointRes=[r[0],r[1],pointX,pointY]
                    ans.append(pointRes)
            pointX+=interval
        pointY-=interval
    return ans

def createRegionByThreePoint(point,extendTimes,interval,res):  #单点坐标
    pointStart=[point[0]-extendTimes*interval,point[1]+extendTimes*interval]
    pointEnd=[point[0]+extendTimes*interval,point[1]-extendTimes*interval]
    pointY,pointX=pointStart[1],pointStart[0]
    ans=[]
    while pointY>=pointEnd[1]:
        pointX=pointStart[0]
        while pointX<=pointEnd[0]:
            for r in res:
                if getDistance(r[:2],[pointX,pointY]) >25 and getDistance(r[2:4],[pointX,pointY]) >25:
                    pointRes=[r[0],r[1],r[2],r[3],pointX,pointY]
                    ans.append(pointRes)
            pointX+=interval
        pointY-=interval
    return ans


def createRegion(point,extendTimes,interval):
    ans=createRegionByFirstPoint(point[:2],extendTimes,interval)
    ans=createRegionBySecondPoint(point[2:4],extendTimes,interval,ans)
    ans=createRegionByThreePoint(point[4:6],extendTimes,interval,ans)
    #print(len(ans))
    return ans
    """
    res=[]
    for r in ans:
        if getDistance(r[:2],r[2:4]) < 32 or getDistance(r[:2],r[4:6])<32 or getDistance(r[2:4],r[4:6])<32:
            continue
        res.append(r)
    return res
    """

def getDistance(num1,num2):
    return pow(abs(num1[0]-num2[0]),2)+pow(abs(num1[1]-num2[1]),2)

def setValueByResource(point,base): #计算放射源辐射强度
    i,j=0,0
    ans=[0,0,0]
    while i<len(base)-2 and j<1:
        #print(point)
        r11=getDistance(point[:2],base[i])
        r12=getDistance(point[2:4],base[i])
        r13=getDistance(point[4:6],base[i])
        c1=base[i][2]
        
        r21=getDistance(point[:2],base[i+1])
        r22=getDistance(point[2:4],base[i+1])
        r23=getDistance(point[4:6],base[i+1])
        c2=base[i+1][2]
        
        r31=getDistance(point[:2],base[i+2])
        r32=getDistance(point[2:4],base[i+2])
        r33=getDistance(point[4:6],base[i+2])
        c3=base[i+2][2]
        
        if r11==0 or r12==0 or r13==0 or r21==0 or r22==0 or r23==0 or r31==0 or r32==0 or r33==0:
            i += 3
            continue
        
        help1=[1/r11,1/r12,1/r13]
        help2=[1/r21,1/r22,1/r23]
        help3=[1/r31,1/r32,1/r33]
        
        help4,help5=[],[]
        help4.append(help1)
        help4.append(help2)
        help4.append(help3)
        help5.append(c1)
        help5.append(c2)
        help5.append(c3)
        #print(help4,help5)
        res = calculate(help4,help5)
        if len(res)!=0:
            ans[0] += res[0]
            ans[1] += res[1]
            ans[2] += res[2]
            j += 1
        i += 1
        
        
    ans[2]/=j
    ans[1]/=j
    ans[0]/=j
    point.extend(ans)
    return point

def setValueByDiff(point,nums):
    val=0
    for num in nums:
        r1=getDistance(point[:2],num)
        r2=getDistance(point[2:4],num)
        r3=getDistance(point[4:6],num)
        preVal=0
        if r1!=0:
            preVal+=point[6]/r1
        else:
            preVal+=point[6]
        if r2!=0:
            preVal+=point[7]/r2
        else:
            preVal+=point[7]
        if r3!=0:
            preVal+=point[8]/r3
        else:
            preVal+=point[8]
        val+=abs(num[2]-preVal)
        #print(num[2],preVal,val)
    point.append(val)
    

def setValue(num,point,extendTimes,interval):
    ans=createRegion(point,extendTimes,interval)
    for point in ans:
        setValueByResource(point,num)
        #print(point)
        setValueByDiff(point,num)
        #print(point)
    ans.sort(key=lambda k:k[9])
    return ans

def predict(num):
    ans=setValue(num,[100.0,100.0,100.0,100.0,100.0,100.0],3,3)
    ans=setValue(num,ans[0][:6],2,2)
    ans=setValue(num,ans[0][:6],2,1)
    #ans=setValue(num,[100.0,100.0,100.0,100.0],4,2)
    #ans=setValue(num,ans[0][:4],3,1)
    #ans=setValue(num,ans[0][:4],4,0.2)
    #ans=setValue(num,ans[0][:4],4,0.1)
    return ans

def calculate(num1,num2):
    a = np.mat(num1)
    b = np.mat(num2).T
    ans = []
    try:
        x = solve(a,b)
        for i in x:
            ans.append(float(i[0]))
    except:
        return ans
    return ans

#num1 = [[1,2,2],[6,8,12],[6,2,3]]
#num2 = [11,71,31]
#calculate(num1,num2)

#point=[0,3,0,0,3,0]
#base=[[0,1,86.4],[1,0,89.1],[0,2,58.15]]
#setValueByResource(point,base)

#createRegion([100,100,100,100,100,100],3,1)
#num.sort(key=lambda k:k[2],reverse = True)
predict(num)

71160
10255
14425


[[94.0,
  107.0,
  98.0,
  93.0,
  106.0,
  100.0,
  120159.34556537865,
  76889.40954525377,
  202832.9045148569,
  59021.02381050602],
 [94.0,
  107.0,
  98.0,
  93.0,
  105.0,
  100.0,
  128553.70121215332,
  77150.79879807902,
  175061.9641986113,
  76541.24831217158],
 [94.0,
  107.0,
  98.0,
  93.0,
  106.0,
  99.0,
  129663.67453534738,
  76448.80319461446,
  183540.08045800452,
  80924.51969476516],
 [95.0,
  107.0,
  98.0,
  93.0,
  106.0,
  100.0,
  140356.22169227246,
  77391.3492151631,
  181506.6693130368,
  90812.58809380536],
 [94.0,
  106.0,
  98.0,
  93.0,
  106.0,
  100.0,
  119163.62783330906,
  77735.37047821489,
  182994.80044930818,
  91450.04749151286],
 [95.0,
  106.0,
  98.0,
  93.0,
  106.0,
  100.0,
  143372.72259274876,
  78437.21567885415,
  153896.79976647944,
  92729.32449816758],
 [94.0,
  107.0,
  98.0,
  93.0,
  105.0,
  99.0,
  138058.28336702086,
  76634.18363036063,
  157457.3652785058,
  94030.8166538435],
 [93.0,
  107.0,
  98.0,
  93.0,
  106.0,


In [40]:
def calculate(num1,num2):
    a = np.mat(num1)
    b = np.mat(num2).T
    ans = []
    try:
        x = solve(a,b)
        for i in x:
            ans.append(float(i[0]))
    except:
        return ans
    return ans

num1 = [[1/164,1/340,1/232],[1/149,1/317,1/221],[1/136,1/296,1/212]]
num2 = [2017,2353,2804]
num3 = [[0,0],[0,0]]
calculate(num3,num2)

[]

In [10]:
def createRegionByFirstPoint(point,extendTimes,interval):  #单点坐标
    pointStart=[point[0]-extendTimes*interval,point[1]+extendTimes*interval]
    pointEnd=[point[0]+extendTimes*interval,point[1]-extendTimes*interval]
    ans=[]
    pointY,pointX=pointStart[1],pointStart[0]
    while pointY>=pointEnd[1]:
        pointX=pointStart[0]
        while pointX<=pointEnd[0]:
            res=[pointX,pointY]
            ans.append(res)
            pointX+=interval
        pointY-=interval
    #print(len(ans))
    return ans

def createRegionBySecondPoint(point,extendTimes,interval,res):  #单点坐标
    pointStart=[point[0]-extendTimes*interval,point[1]+extendTimes*interval]
    pointEnd=[point[0]+extendTimes*interval,point[1]-extendTimes*interval]
    pointY,pointX=pointStart[1],pointStart[0]
    ans=[]
    while pointY>=pointEnd[1]:
        pointX=pointStart[0]
        while pointX<=pointEnd[0]:
            for r in res:
                if r[0]==pointX and r[1]==pointY:
                    continue;
                pointRes=[r[0],r[1],pointX,pointY]
                ans.append(pointRes)
            pointX+=interval
        pointY-=interval
    #print(len(ans))
    return ans

def createRegion(point,extendTimes,interval):
    ans=createRegionByFirstPoint(point[:2],extendTimes,interval)
    ans=createRegionBySecondPoint(point[2:4],extendTimes,interval,ans)
    res=[]
    for r in ans:
        if getDistance(r[:2],r[2:4])>18:
            res.append(r)
    #print(res)
    return res

def getDistance(num1,num2):
    return pow(abs(num1[0]-num2[0]),2)+pow(abs(num1[1]-num2[1]),2)

def setValueByResource(point,base): #计算放射源辐射强度
    i,j=0,0
    ans=[0,0]
    while i<len(base)-1 and j<20:
        r1=getDistance(point[:2],base[i])
        r2=getDistance(point[2:4],base[i])
        c1=base[i][2]
        r3=getDistance(point[:2],base[i+1])
        r4=getDistance(point[2:4],base[i+1])
        c2=base[i+1][2]
        if r1*r4-r2*r3!=0:
            num1=(r1*r2*r4*c1-r2*r3*r4*c2)/(r1*r4-r2*r3)
            num0=(r1*r2*c1-r1*num1)/r2
            #print(num0,num1,point)
            #if num1>0 and num0>0:
            j+=1
            ans[1]+=num1
            ans[0]+=num0
        i+=1
    ans[1]/=j
    ans[0]/=j
    """
    while r1*r4-r2*r3==0:
        i+=1;
        r1=getDistance(point[:2],base[i])/(1-0.0.009)
        r2=getDistance(point[2:4],base[i])/(1-0.011)
        c1=base[i][2]
        r3=getDistance(point[:2],base[i+1])/(1-0.01)
        r4=getDistance(point[2:4],base[i+1])/(1-0.012)
    """
    point.extend(ans)
    return ans

def setValueByDiff(point,nums):
    val=0
    for num in nums:
        r1=getDistance(point[:2],num)
        r2=getDistance(point[2:4],num)
        preVal=0
        if r1!=0:
            preVal+=point[4]/r1
        else:
            preVal+=point[4]
        if r2!=0:
            preVal+=point[5]/r2
        else:
            preVal+=point[5]
        val+=abs(num[2]-preVal)
        #print(num[2],preVal,val)
    point.append(val)

def setValue(num,point,extendTimes,interval):
    ans=createRegion(point,extendTimes,interval)
    for point in ans:
        setValueByResource(point,num)
        #print(point)
        setValueByDiff(point,num)
        #print(point)
    ans.sort(key=lambda k:k[6])
    return ans

def predict(num):
    ans=setValue(num,[100.0,100.0,100.0,100.0],4,2)
    ans=setValue(num,ans[0][:4],3,1)
    #ans=setValue(num,ans[0][:4],4,0.2)
    #ans=setValue(num,ans[0][:4],4,0.1)
    return ans[0]
    

#createRegion([100,100,100,100],2,30)
#num.sort(key=lambda k:k[2],reverse = True)
#num=num[:3]
#print(num)
predict(num)

[102.0,
 105.0,
 94.0,
 96.0,
 161077.55407242867,
 180485.71625187248,
 14595.368759138897]